# Coroutines

Pythons async/await coroutines were the initial motivation for **Co**HDL. Coroutines are functions that can suspend and resume their execution. CoHDL translates coroutines into VHDL state machines. This translation process is completely deterministic and makes it possible to describe sequential processes clock cycle accurate.

## async/await

The `async` keyword turns functions into coroutines. Only `asnyc def` functions can use `await` expressions in the function body.

For CoHDL there are two different types of await expression:

* awaiting primitive expressions

    When the argument of `await` is a Signal/Variable/Temporary, the coroutines execution is suspended, until that argument becomes truthy (non-zero). Each wait takes at least one clock cycle event when the argument is true. `await` expressions define wait states, the code between two awaits is executed once when transitioning from one wait to the next. These primitive awaits are the building blocks for more complex sequential processes.
* awaiting coroutine functions

    When the argument of `await` is itself a coroutine, CoHDL treats that expression similar to a normal function call. The function body - that may contain nested `await` expressions - is translated and inlined at the call site.

In [1]:
from cohdl import Entity, Port, Bit, Unsigned
from cohdl import std

async def coro_fn(step, output):
    await step
    output <<= "01"
    # the awaited expression is evaluated
    # every clock cycle. This is different
    # from the normal Python behavior where
    # the expression is evaluated once and only
    # the resulting object is awaited.
    await (~step)
    output <<= "10"
    await step

class Counter(Entity):
    clk = Port.input(Bit)
    step = Port.input(Bit)

    output = Port.output(Unsigned[2])

    def architecture(self):
        clk = std.Clock(self.clk)

        @std.sequential(clk)
        async def coroutine_process():
            await self.step
            self.output <<= "00"

            # use the coro_fn coroutine
            # this call will take multiple clock cycles
            await coro_fn(self.step, self.output)
            self.output <<= "11"

print(std.VhdlCompiler.to_string(Counter))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Counter is
  port (
    clk : in std_logic;
    step : in std_logic;
    output : out unsigned(1 downto 0)
    );
end Counter;


architecture arch_Counter of Counter is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_output : unsigned(1 downto 0);
  type state_coroutine_process is (state_0, state_1, state_2, state_3);
  signal s_coroutine_process : state_coroutine_process := state_0;
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  output <= buffer_output;
  

  coroutine_process: process(clk)
    variable temp : std_logic;
  begin
    if rising_edge(clk) then
      case s_coroutine_process is
        when state_0 =>
          if step = '1' then
            buffer_output <= unsigned(std_logic_vector'("00"));
            s_coroutine_process <= state

## while loops

While loops are used to describe repeating sequences of states. Like `await`-expressions `while`-loops can only be used in `async` functions. In CoHDL, the body of while-loops translated into states, and transitions back to the beginning of the loop are added to all states that reach the end of the loop body. 

Each occurrence of `while` starts a new state - even if the condition is false it takes at least one clock cycle to step over the loop.

In [2]:
from cohdl import Entity, Port, Bit, Unsigned, BitVector, Signal
from cohdl import std

class SerialReceiver(Entity):
    clk = Port.input(Bit)
    
    start = Port.input(Bit)
    input = Port.input(Bit)

    new_output = Port.output(Bit, default=False)
    output = Port.output(BitVector[8])

    def architecture(self):
        clk = std.Clock(self.clk)

        @std.sequential(clk)
        async def coroutine_process():
            await self.start

            cnt = Signal[Unsigned[3]](7)
            buffer = Signal[BitVector[8]]()

            # transition happens in this line
            # the while condition will be evaluated
            # in the next clock cycle
            while cnt:
                buffer[7:1] <<= buffer[6:0]
                buffer[0] <<= self.input
                cnt <<= cnt - 1
                # cohdl inserts a transition back
                # to the loop start in this line
            
            self.output <<= buffer
            self.new_output ^= True

print(std.VhdlCompiler.to_string(SerialReceiver))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity SerialReceiver is
  port (
    clk : in std_logic;
    start : in std_logic;
    input : in std_logic;
    new_output : out std_logic;
    output : out std_logic_vector(7 downto 0)
    );
end SerialReceiver;


architecture arch_SerialReceiver of SerialReceiver is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_new_output : std_logic := '0';
  signal buffer_output : std_logic_vector(7 downto 0);
  signal sig : unsigned(2 downto 0);
  type state_coroutine_process is (state_0, state_1);
  signal s_coroutine_process : state_coroutine_process := state_0;
  signal sig_1 : std_logic_vector(7 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  new_output <= buffer_new_output;
  output <= buffer_output;
  

  coroutine_process: process(clk)
    vari

While loops can also be used as an alternative to await expressions, with the additional ability to customize signal states during the wait period. In CoHDL awaiting a Signal/Temporary is effectively syntactic sugar for a while loop with an empty body.

In [3]:
from cohdl import Entity, Port, Bit, Unsigned
from cohdl import std

class Counter(Entity):
    clk = Port.input(Bit)
    step = Port.input(Bit)

    output = Port.output(Unsigned[2])

    def architecture(self):
        clk = std.Clock(self.clk)

        @std.sequential(clk)
        async def coroutine_process():
            # wait until step becomes truthy
            # do nothing else
            await self.step

            self.output <<= "00"

            # wait until step becomes truthy
            # do nothing else (equivalent to first await expression)
            while ~self.step:
                pass
            
            self.output <<= "01"

            # wait until step becomes truthy
            # and define the state of output while waiting
            while ~self.step:
                self.output <<= "10"
            
            self.output <<= "11"

print(std.VhdlCompiler.to_string(Counter))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Counter is
  port (
    clk : in std_logic;
    step : in std_logic;
    output : out unsigned(1 downto 0)
    );
end Counter;


architecture arch_Counter of Counter is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_output : unsigned(1 downto 0);
  type state_coroutine_process is (state_0, state_1, state_2);
  signal s_coroutine_process : state_coroutine_process := state_0;
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  output <= buffer_output;
  

  coroutine_process: process(clk)
    variable temp : std_logic;
    variable temp_1 : std_logic;
  begin
    if rising_edge(clk) then
      case s_coroutine_process is
        when state_0 =>
          if step = '1' then
            buffer_output <= unsigned(std_logic_vector'("00"));
            s_co

## coroutines and classes

Coroutines can be members of classes. This is used by the SerialTransmitter class to define a sequential send logic that operates on the transmit signal of a serial interface. To keep the example simple there is no synchronization logic. The same basic structure could also be used for more complex interfaces such as AXI or Wishbone.

In [4]:
from cohdl import Entity, Port, Bit, Unsigned, BitVector, Signal
from cohdl import std

# The SerialTransmitter class wraps a single bit
# transmit signal and defines a coroutine method
# send, that serializes and sends one byte of data
class SerialTransmitter:
    def __init__(self, tx: Signal[Bit]):
        self._tx = tx
    
    # serialize data and send it via the single bit tx signal
    # this coroutine will return after 8 clock cycles
    async def send(self, data: Signal[BitVector[8]]):
        cnt = Signal[Unsigned[4]](8)
        buffer = Signal(data)

        while cnt:
            cnt <<= cnt - 1
            self._tx <<= buffer[0]
            buffer[6:0] <<= buffer[7:1]


class TransmitterExample(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    data = Port.input(BitVector[8])
    tx = Port.output(Bit)

    def architecture(self):
        clk = std.Clock(self.clk)
        reset = std.Reset(self.reset)

        transmitter = SerialTransmitter(self.tx)

        @std.sequential(clk, reset)
        async def proc_use_transmitter():
            # perform the transmitter send operation
            # this coroutine call will take 8 clock cycles
            await transmitter.send(self.data)


print(std.VhdlCompiler.to_string(TransmitterExample))


library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity TransmitterExample is
  port (
    clk : in std_logic;
    reset : in std_logic;
    data : in std_logic_vector(7 downto 0);
    tx : out std_logic
    );
end TransmitterExample;


architecture arch_TransmitterExample of TransmitterExample is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_tx : std_logic;
  type state_proc_use_transmitter is (state_0, state_1);
  signal s_proc_use_transmitter : state_proc_use_transmitter := state_0;
  signal sig : unsigned(3 downto 0);
  signal sig_1 : std_logic_vector(7 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  tx <= buffer_tx;
  

  proc_use_transmitter: process(clk)
    variable temp : unsigned(3 downto 0);
  begin
    if rising_edge(clk) then
      if reset = '1' then
        s_proc_use_transm

## alternative implementation

The following example show an alternative implementation of the SerialTransmitter class. It defines a separate sequential context for the serialization process, where the send method is not longer a coroutine. Instead, the send method forwards the given data alongside a start signal to the other process.

In [5]:
from cohdl import Entity, Port, Bit, Unsigned, BitVector, Signal
from cohdl import std

# In this version of SerialTransmitter
# the serialization logic is defined in its own
# sequential context. send only transfers data
# to that context and sets a start signal.
class SerialTransmitter:
    def __init__(self, clk, reset, tx):
        # define local signals
        self._start = Signal[bool](False)
        self._data = Signal[BitVector[8]]()
        self._ready = Signal[bool](False)

        @std.sequential(clk, reset)
        async def proc_serial_transmitter():
            # wait for start signal
            while not self._start:
                self._ready <<= True
            self._ready <<= False

            # create a local copy of the data to send
            buffer = Signal(self._data)
            cnt = Signal[Unsigned[4]](8)

            while cnt:
                cnt <<= cnt - 1
                tx.next = buffer[0]
                buffer[6:0] <<= buffer[7:1]
    
    def ready(self):
        return self._ready
    
    # not a coroutine, this method
    # starts the transmit sequence in the parallel
    # sequential context
    def send(self, data):
        # assert, that send only gets called when
        # the transmitter is ready
        assert self._ready
        self._data <<= data
        self._start ^= True


class TransmitterExample(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    data = Port.input(BitVector[8])
    tx = Port.output(Bit)

    def architecture(self):
        clk = std.Clock(self.clk)
        reset = std.Reset(self.reset)

        transmitter = SerialTransmitter(clk, reset, self.tx)

        @std.sequential(clk, reset)
        async def proc_use_transmitter():
            await transmitter.ready()
            transmitter.send(self.data)
            # this process can do some other work for up to 8 clock cycles
            # before the transmitter becomes ready again


print(std.VhdlCompiler.to_string(TransmitterExample))


library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity TransmitterExample is
  port (
    clk : in std_logic;
    reset : in std_logic;
    data : in std_logic_vector(7 downto 0);
    tx : out std_logic
    );
end TransmitterExample;


architecture arch_TransmitterExample of TransmitterExample is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_tx : std_logic;
  signal sig : boolean := false;
  type state_proc_serial_transmitter is (state_0, state_1);
  signal s_proc_serial_transmitter : state_proc_serial_transmitter := state_0;
  signal sig_1 : boolean := false;
  signal sig_2 : std_logic_vector(7 downto 0);
  signal sig_3 : std_logic_vector(7 downto 0);
  signal sig_4 : unsigned(3 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  tx <= buffer_tx;
  

  proc_serial_transmitter: process(clk)
 